In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
import tensorflow.keras
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax

In [ ]:
import os
import shutil

# Paths for the two datasets
dataset_200_path = '/kaggle/input/yoga-pose-classification/YogaPoses'
dataset_100_path = '/kaggle/input/yoga-poses-dataset/DATASET'

# Final combined folder path
combined_dataset_path = '/kaggle/working/combined_dataset'

# Pose folder names (standardized to lowercase)
pose_folders = ['downdog', 'goddess', 'plank', 'tree', 'warrior2']

# Create the combined dataset folder with 5 subfolders (one for each pose)
if not os.path.exists(combined_dataset_path):
    os.makedirs(combined_dataset_path)

for pose in pose_folders:
    pose_folder = os.path.join(combined_dataset_path, pose)
    if not os.path.exists(pose_folder):
        os.makedirs(pose_folder)

# Function to copy images from a dataset folder to the combined dataset
def copy_images(dataset_path, combined_dataset_path, pose_folders):
    for pose_folder in pose_folders:
        # Handle both upper and lower case folder names
        src_folder_200 = os.path.join(dataset_path, pose_folder.capitalize())
        src_folder_100 = os.path.join(dataset_path, pose_folder.lower())

        # Check if the folder exists and copy files from it
        if os.path.exists(src_folder_200):
            for filename in os.listdir(src_folder_200):
                src_file = os.path.join(src_folder_200, filename)
                dst_file = os.path.join(combined_dataset_path, pose_folder, filename)
                shutil.copy(src_file, dst_file)
        elif os.path.exists(src_folder_100):
            for filename in os.listdir(src_folder_100):
                src_file = os.path.join(src_folder_100, filename)
                dst_file = os.path.join(combined_dataset_path, pose_folder, filename)
                shutil.copy(src_file, dst_file)

# Copy images from Dataset_200
copy_images(dataset_200_path, combined_dataset_path, pose_folders)

# Copy images from Dataset_100 (inside 'train' and 'test' folders)
copy_images(os.path.join(dataset_100_path, 'TRAIN'), combined_dataset_path, pose_folders)
copy_images(os.path.join(dataset_100_path, 'TEST'), combined_dataset_path, pose_folders)

print("Images from both datasets combined into 5 pose subfolders.")


In [ ]:
import mediapipe as mp
import cv2
import os
import numpy as np

# Initialize MediaPipe Pose.
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils  # Utility to draw landmarks and connections
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Main input and output folder paths
input_main_folder = '/kaggle/working/combined_dataset'
output_main_folder = '/kaggle/working/skeleton_combined'

# Create output main folder if it doesn't exist
os.makedirs(output_main_folder, exist_ok=True)

# Loop through each subfolder (pose type)
for subfolder in os.listdir(input_main_folder):
    input_folder = os.path.join(input_main_folder, subfolder)
    output_folder = os.path.join(output_main_folder, subfolder)
    
    # Check if the subfolder is a directory (pose folder)
    if os.path.isdir(input_folder):
        # Create output folder for each pose if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)

        # Process each image in the input folder
        for filename in os.listdir(input_folder):
            if filename.endswith(('.png', '.jpg', '.jpeg')):
                # Read the image
                image_path = os.path.join(input_folder, filename)
                image = cv2.imread(image_path)
                image_height, image_width, _ = image.shape

                # Convert the image to RGB
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Process the image and find the pose
                result = pose.process(image_rgb)

                # Create a blank numpy array with zeros
                skeleton_image = np.zeros((image_height, image_width, 3), dtype=np.uint8)

                # Draw the skeleton on the blank image
                if result.pose_landmarks:
                    mp_drawing.draw_landmarks(
                        skeleton_image, 
                        result.pose_landmarks, 
                        mp_pose.POSE_CONNECTIONS,  # This connects the keypoints to form the skeleton
                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=5, circle_radius=5),
                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
                    )

                # Save the image with the skeleton
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, skeleton_image)

print("Processing complete.")


In [ ]:
#split test train combined data
import os
import shutil
import numpy as np

def split_data(src_dir, dst_dir, train_ratio=0.8):
    # Ensure destination directories exist
    os.makedirs(os.path.join(dst_dir, 'train'), exist_ok=True)
    os.makedirs(os.path.join(dst_dir, 'test'), exist_ok=True)
    
    # List of pose folders
    pose_folders = ['downdog', 'goddess', 'plank', 'tree', 'warrior2']
    
    for pose in pose_folders:
        # Paths
        src_pose_dir = os.path.join(src_dir, pose)
        train_pose_dir = os.path.join(dst_dir, 'train', pose)
        test_pose_dir = os.path.join(dst_dir, 'test', pose)
        
        # Create subdirectories for train and test
        os.makedirs(train_pose_dir, exist_ok=True)
        os.makedirs(test_pose_dir, exist_ok=True)
        
        # Get all images in the pose directory
        all_images = os.listdir(src_pose_dir)
        
        # Shuffle the images
        np.random.shuffle(all_images)
        
        # Split indices
        split_idx = int(len(all_images) * train_ratio)
        
        # Split images into train and test
        train_images = all_images[:split_idx]
        test_images = all_images[split_idx:]
        
        # Move images to respective directories
        for img in train_images:
            shutil.move(os.path.join(src_pose_dir, img), os.path.join(train_pose_dir, img))
        for img in test_images:
            shutil.move(os.path.join(src_pose_dir, img), os.path.join(test_pose_dir, img))

# Define source and destination directories
src_directory = '/kaggle/working/skeleton_combined'
dst_directory = '/kaggle/working/split_data'

# Perform the split
split_data(src_directory, dst_directory)


In [ ]:
train_dir = '/kaggle/working/split_data/train' #directory with training images
test_dir = '/kaggle/working/split_data/test' #directory with testing images

In [ ]:
import os
from collections import Counter
from sklearn.utils import class_weight
import numpy as np

# Define the path to the training data
train_dir = '/kaggle/working/split_data/train'

# List all the class subfolders
class_names = os.listdir(train_dir)

# Initialize a list to store the labels (class indices) of all images
train_labels = []

# Go through each class folder and count the images
for class_index, class_name in enumerate(class_names):
    class_path = os.path.join(train_dir, class_name)
    num_images = len(os.listdir(class_path))
    # Append the class index `num_images` times to the train_labels list
    train_labels.extend([class_index] * num_images)

# Now `train_labels` contains the class indices of all images in the training set
# Compute the class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),  # The unique class labels
    y=train_labels  # The list of labels
)

# Convert the class weights to a dictionary
class_weights_dict = dict(enumerate(class_weights))
print(class_weights_dict)  # Example: {0: 0.8, 1: 1.2, 2: 0.5, 3: 1.1, 4: 1.4}



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Enhanced data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,            # Use 20% of data for validation
    width_shift_range=0.1,            # Randomly shift images horizontally
    height_shift_range=0.1,           # Randomly shift images vertically
    horizontal_flip=True,             # Randomly flip images horizontally
    rotation_range=15,                # Randomly rotate images in the range (degrees)
    zoom_range=0.1,                   # Randomly zoom in on images
    shear_range=0.1,                  # Randomly apply shearing transformations
    brightness_range=[0.8, 1.2],     # Randomly adjust brightness
    fill_mode='nearest'               # Fill mode for newly created pixels
)

# Data augmentation for testing is not needed, so just rescaling
test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(75, 75),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    directory=train_dir,  # This should be the same as train_dir since validation is from training data
    target_size=(75, 75),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16,
    subset='validation'
)

# Create test generator
test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(75, 75),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16
)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding = 'Same', input_shape=(75, 75, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    # tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),
    # tf.keras.layers.MaxPooling2D(2,2),
    # tf.keras.layers.Dropout(0.25),
    # tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),
    # tf.keras.layers.MaxPooling2D(2,2),
    # tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
optimizer = Adam(learning_rate=0.0005)
model.compile(loss='categorical_crossentropy',
              optimizer = optimizer,
              metrics=['accuracy'])
epochs = 100 
batch_size = 32

In [ ]:
model.summary()

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define the checkpoint callback to save the model's weights
checkpoint = ModelCheckpoint('model_checkpoint.weights.h5', monitor='val_loss', save_best_only=False, save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Compile the model
optimizer = Adam(learning_rate=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with the checkpoint callback
history = model.fit(train_generator, 
                    epochs=epochs, 
                    validation_data=validation_generator, 
                    callbacks=[checkpoint,early_stopping,lr_scheduler],
                    class_weight=class_weights_dict)


In [ ]:
# Load the model's weights from the checkpoint
model.load_weights('model_checkpoint.weights.h5')
# Save the entire model
model.save('combined_cnn.keras')  # Save the complete model

# Continue training from the last saved checkpoint
# history = model.fit(train_generator, 
#                     epochs=epochs, 
#                     validation_data=validation_generator, 
#                     callbacks=[checkpoint])


In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
# import os
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# import numpy as np
# from sklearn.metrics import accuracy_score

# # Directory containing the test folders
# test_dir = '/kaggle/working/output_pic_withline/test'  # Update with your test dataset path that is skeletonised
# pose_folders = ['downdog', 'goddess', 'plank', 'tree', 'warrior2']  # Update with your actual folder names

# pose_map = {0: 'downdog', 1: 'goddess', 2: 'plank', 3: 'tree', 4: 'warrior2'}

# # Initialize lists to store true labels and predictions
# true_labels = []
# predictions = []
# predicted_pose_names = []

# # Load and preprocess images
# for label, folder in enumerate(pose_folders):
#     folder_path = os.path.join(test_dir, folder)
#     for img_name in os.listdir(folder_path):
#         img_path = os.path.join(folder_path, img_name)
#         img = load_img(img_path, target_size=(75, 75))  # Same size as your training images
        
#         # Convert the image to a numpy array and normalize it
#         img_array = img_to_array(img)
#         img_array = img_array / 255.0  # Rescale the image
        
#         # Add an additional dimension for batch size (as the model expects a batch of images)
#         img_array = np.expand_dims(img_array, axis=0)
        
#         # Predict the class of the image
#         pred = model.predict(img_array)
        
#         # Get the predicted class index
#         predicted_class_index = np.argmax(pred, axis=1)[0]
        
#         # Append true label and prediction
#         true_labels.append(label)
#         predictions.append(predicted_class_index)
        
#         predicted_pose_name = pose_map[predicted_class_index]
#         predicted_pose_names.append(predicted_pose_name)

# # Calculate accuracy
# accuracy = accuracy_score(true_labels, predictions)
# print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
# model.save('/kaggle/working/cnn_landmark_model.keras')  # Update the path and filename as needed

In [ ]:
# from tensorflow import keras

# # Load the saved model
# m1 = keras.models.load_model('/kaggle/input/cnn_landmark/tensorflow2/default/1/cnn_landmark_model.keras')
